<a href="https://colab.research.google.com/github/simpleParadox/Private-RE/blob/main/project_622.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install transformers --quiet
!pip install pyvacy --quiet

In [6]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.nn.functional as F

# Import the transformers library for the retrieving the BERT embeddings.
import transformers
from transformers import BertModel, BertTokenizer


# Import pyvacy for privacy preserving optimizers.
from pyvacy import optim, analysis

# Import scikit-learn packages.
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.utils import shuffle
from sklearn.metrics import f1_score


# Import scientific computing python packages.
import pandas as pd
import numpy as np      
import matplotlib.pyplot as plt

# Additional packages.
from google.colab import drive
from tqdm import tqdm
import csv
from typing import List

In [ ]:
# optimizer = optim.DPSGD(params=model.parameters(), **training_parameters) 
# epsilon = analysis.epsilon(**training_parameters)
# loss_function = ... 

### Read in Erin's tabular data and preprocess it.

In [7]:
drive.mount('/content/drive')
relations_path = '/content/drive/MyDrive/CMPUT 622 project/data/tabular_data/Input_all_29_relation.tsv'

train_data = pd.read_csv(relations_path, encoding='utf-8', sep = '\t')

print(train_data.shape[0])

Mounted at /content/drive
14103113


In [8]:
train_data.isnull().sum()

table_caption          13436015
table_section_title      429931
headers                   11639
entity pair                   0
relation                      0
dtype: int64

In [9]:
train_data.head()

,table_caption,table_section_title,headers,entity pair,relation
0,Recipients,Recipients,Name of the recipient,Nishan-e-Haider Saif Ali Janjua Saif Ali Janj...,award.award_nominated_work.award_nominations.....
1,Recipients,Recipients,Name of the recipient,Nishan-e-Haider Raja Muhammad Sarwar Raja Muh...,award.award_nominated_work.award_nominations.....
2,Recipients,Recipients,Name of the recipient,Nishan-e-Haider Tufail Mohammad Tufail Mohammad,award.award_nominated_work.award_nominations.....
3,Recipients,Recipients,Name of the recipient,Nishan-e-Haider Raja Aziz Bhatti Raja Aziz Bh...,award.award_nominated_work.award_nominations.....
4,Recipients,Recipients,Name of the recipient,Nishan-e-Haider Rashid Minhas Rashid Minhas,award.award_nominated_work.award_nominations.....


In [10]:
train_data.fillna("", inplace = True)

# Shuffle data so that there is a higher chance of the train and test data being from the same distribution.
train_data = shuffle(train_data, random_state = 1)


labels = train_data.iloc[:,-1].values
sentences = train_data.iloc[:,:-1].values.tolist()

sentences = [' '.join(sent).strip() for sent in sentences]

label = preprocessing.LabelEncoder()
y = label.fit_transform(train_data['relation'])
label_mappings = integer_mapping = {i: l for i, l in enumerate(label.classes_)}

In [11]:
train_data[:2]

,table_caption,table_section_title,headers,entity pair,relation
2813793,,Selected filmography,Title,Hedda Hopper Miami Miami 1924 film,film.performance.actor..film.performance.film
6273720,,1992–93 squad,Born,Turkey Turkey Ankara Ankara,location.location.contains


In [15]:
train_data.isnull().sum()

table_caption          0
table_section_title    0
headers                0
entity pair            0
relation               0
dtype: int64

###**Read Sententence-level Data**

In [16]:
train_directory_path = '/content/drive/MyDrive/CMPUT 622 project/data/semeval/train.txt'
test_directory_path = '/content/drive/MyDrive/CMPUT 622 project/data/semeval/test.txt'

In [17]:
relation_to_id = [
    "other", 
    "Entity-Destination(e1,e2)",
    "Cause-Effect(e2,e1)",        
    "Member-Collection(e2,e1)",      
    "Entity-Origin(e1,e2)",        
    "Message-Topic(e1,e2)",        
    "Component-Whole(e2,e1)",       
    "Component-Whole(e1,e2)",       
    "Instrument-Agency(e2,e1)",     
    "Product-Producer(e2,e1)",     
    "Content-Container(e1,e2)",     
    "Cause-Effect(e1,e2)",          
    "Product-Producer(e1,e2)",       
    "Content-Container(e2,e1)",    
    "Entity-Origin(e2,e1)",          
    "Message-Topic(e2,e1)",        
    "Instrument-Agency(e1,e2)",       
    "Member-Collection(e1,e2)",      
    "Entity-Destination(e2,e1)"]    

In [18]:
def convertText_csv(path):
  output: List[List[str]] = []

  with open(path) as file:
    lines = file.read()
    lines =  lines.splitlines()

  for line in lines:
    line = line.strip()
    input = line.split(sep="\t")
    entity1 = input[0]
    entity2 = input[1]
    relation = relation_to_id.index(input[2])
    sentence = input[3]

    sentence = sentence.replace('<e1>', '')
    sentence = sentence.replace('<e2>', '')
    sentence = sentence.replace('</e1>', '')
    sentence = sentence.replace('</e2>', '')
    
    output.append([sentence, entity1, entity2, relation])
  return output

In [19]:
def writeOutput(output, path):
  with open(path, 'w', newline='') as f:
    writer = csv.writer(f, delimiter='\t')
    writer.writerow(["sentence", "entity1", "entity2", "relation"])
    for i in output:
      writer.writerow(i)

In [51]:
def loadSemEvalDateset(outputFilename):
  writeOutput(convertText_csv(train_directory_path), outputFilename)
  data = pd.read_csv(outputFilename, encoding='utf-8', sep = '\t')

  #data = shuffle(data, random_state = 1) 
  print(data[:1])

  labels = data.iloc[:,-1].values
  features = data.iloc[:,:-1].values.tolist()
  sentences = [' '.join(i).strip() for i in features]
  return sentences, labels

In [52]:
x_train, y_train = loadSemEvalDateset("train.tsv")

                                            sentence        entity1   entity2  \
0  The system as described above has its greatest...  configuration  elements   

   relation  
0         6  


In [53]:
x_train[0]

'The system as described above has its greatest application in an arrayed  configuration  of antenna  elements  . configuration elements'

In [54]:
x_test, y_test = loadSemEvalDateset("test.tsv")

                                            sentence        entity1   entity2  \
0  The system as described above has its greatest...  configuration  elements   

   relation  
0         6  


In [57]:
x_test[0] # probably test and train files have same data

'The system as described above has its greatest application in an arrayed  configuration  of antenna  elements  . configuration elements'

In [56]:
y_test.shape

(8000,)

In [ ]:
y_train = torch.tensor(y_train)
y_train

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


tensor([6, 0, 8,  ..., 1, 0, 9])

## Initialize the pretrained BERT model (uncased) and the respective tokenizer.

### NOTE: We might need to tokenize and encode everything before running the model.

### Putting the tokenizer into a function.

In [ ]:
def bert_tokenize(texts, tokenizer):
    all_encoded_inputs = []
    # print("texts: ", texts)
    
    for i in tqdm(range(len(texts))):
        text = texts[i]
        # text = text[:max_len-2]
        # input_sequence = ["[CLS]"] + text + ["[SEP]"]
        # pad_len = max_len-len(input_sequence)""
        encoded_input = tokenizer(text, return_tensors='pt', padding="max_length", max_length=50, truncation=True)
        all_encoded_inputs.append(encoded_input)
        
    return all_encoded_inputs

### Get BERT embeddings

In [ ]:
def get_bert_embeds_from_tokens(bert_model, encoded_inputs):
    all_bert_embeds = []
    for i in tqdm(range(len(encoded_inputs))):
        encoded_input = encoded_inputs[i]
        # print("encoded input: ", type(encoded_input))
        outputs = bert_model(**encoded_input)
        # pooler_output = outputs['pooler_output']  # We don't need this for our analysis.
        hidden_states = outputs['last_hidden_state']
        all_bert_embeds.append(np.squeeze(hidden_states.detach().numpy()))
    return all_bert_embeds

### Define the BertTokenizer and the BertModel from the transformers library.

In [ ]:
# Define the BertModel and the BertTokenizer
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', model_max_length=50, padding_side='right')
bert_model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Encode the inputs and store them so that we don't have re-encode everytime we run the model.

In [ ]:
# First get the train test splits on the sentences and the labels.
seeds = [0]
all_train_last_hidden_states = []
all_test_last_hidden_states = []
# NOTE: Since colab is running out of memory, you can process this in batches and then concatenate the results. See if this works. If not, then move to Compute Canada.
for seed in seeds:
    X_train_texts, X_test_texts, y_train_classes, y_test_classes = train_test_split(sentences, y, random_state=seed, test_size=0.2)

    
    # Now do the tokenization and the encoding process.
    train_tokens = bert_tokenize(X_train_texts, bert_tokenizer)
    test_tokens = bert_tokenize(X_test_texts, bert_tokenizer)

    # # Now get the encodings from BERT. NOTE: The get_bert_embeds_from_tokens function only returns the last_hidden_state vector for the input.
    last_hidden_states_train = get_bert_embeds_from_tokens(bert_model, train_tokens)
    last_hidden_states_test = get_bert_embeds_from_tokens(bert_model, test_tokens)


    # # Store the hidden states
    all_train_last_hidden_states.append(last_hidden_states_train)
    all_test_last_hidden_states.append(last_hidden_states_test)

np.savez_compressed(f"train_embeds_seed_{seed}.npz", all_train_last_hidden_states)
np.savez_compressed(f"test_embeds_seed_{seed}.npz", all_test_last_hidden_states)

 19%|█▉        | 2174495/11282490 [31:19<3955:16:57,  1.56s/it]

In [ ]:
while True:
    print("hello world")

In [ ]:
data = np.load("/content/test_embeds_seed_0.npz", allow_pickle=True)['arr_0'][0]

In [ ]:
data.shape

(2, 50, 768)

## Implement the model

In [ ]:
class erin_model(nn.Module):
    def __init__(self):
        super(erin_model,self).__init__()

        # Just add one LSTM unit as the model followed by a fully connected layer and then a softmax.

        self.lstm = nn.LSTM(input_size=)
